<a href="https://colab.research.google.com/github/zxnb01/Data-Janitor/blob/main/clean_dirty_cafe_sales2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)


In [25]:

FILE_PATH = "data/dirty_cafe_sales.csv"

df = pd.read_csv(FILE_PATH)
print("Initial shape:", df.shape)
df.head()


Initial shape: (10000, 8)


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [26]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

df.columns


Index(['transaction_id', 'item', 'quantity', 'price_per_unit', 'total_spent', 'payment_method', 'location', 'transaction_date'], dtype='object')

In [27]:
if 'transaction_id' not in df.columns:
    # If the column renaming from cell D7KS71pXwilA hasn't applied, apply it now.
    # This handles cases where cells might be run out of order or state is inconsistent.
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
    )

df = df.drop_duplicates(subset="transaction_id")
print("After removing duplicate transaction IDs:", df.shape)


After removing duplicate transaction IDs: (10000, 8)


In [28]:
## Drop Rows with Missing Critical Fields
df = df.dropna(subset=["transaction_id", "item", "transaction_date"])
print("After dropping critical nulls:", df.shape)


After dropping critical nulls: (9514, 8)


Cleaning Categorical Columns

In [29]:
categorical_cols = ["item", "payment_method", "location"]

for col in categorical_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.strip()
        .str.title()
    )

    df[col] = df[col].replace(
        ["Nan", "None", "Null", "Unknown", "[Null]"],
        "Unknown"
    )

Clean Numerical Columns

In [30]:
## quantity column

df["quantity"] = pd.to_numeric(df["quantity"], errors="coerce")
df = df[df["quantity"] > 0]

# Median imputation
df["quantity"] = df["quantity"].fillna(df["quantity"].median()).astype(int)


In [31]:
## Price per unit

df["price_per_unit"] = pd.to_numeric(df["price_per_unit"], errors="coerce")

# Fill missing prices using median price per item
df["price_per_unit"] = df.groupby("item")["price_per_unit"].transform(
    lambda x: x.fillna(x.median())
)

# Fallback (in case entire item group is null)
df["price_per_unit"] = df["price_per_unit"].fillna(df["price_per_unit"].median())

# Remove invalid prices
df = df[df["price_per_unit"] > 0]


In [32]:
df["total_spent"] = df["quantity"] * df["price_per_unit"]


In [33]:
df["transaction_date"] = pd.to_datetime(
    df["transaction_date"],
    errors="coerce"
)

df = df.dropna(subset=["transaction_date"])


Outlier Removal (IQR – conservative)

In [34]:
def remove_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    return data[
        (data[column] >= Q1 - 1.5 * IQR) &
        (data[column] <= Q3 + 1.5 * IQR)
    ]

for col in ["quantity", "price_per_unit", "total_spent"]:
    df = remove_outliers_iqr(df, col)

print("After outlier removal:", df.shape)


After outlier removal: (8546, 8)


In [35]:
df["day"] = df["transaction_date"].dt.day
df["month"] = df["transaction_date"].dt.month
df["weekday"] = df["transaction_date"].dt.weekday


In [36]:
assert df["quantity"].min() > 0
assert df["price_per_unit"].min() > 0
assert (df["total_spent"] == df["quantity"] * df["price_per_unit"]).all()
assert df["transaction_id"].is_unique

print("All sanity checks passed.")


All sanity checks passed.


In [37]:
OUTPUT_FILE = "output/clean_cafe_sales.csv"
df = df.reset_index(drop=True)
df.to_csv(OUTPUT_FILE, index=False)

print("Cleaning complete")
print("Final dataset shape:", df.shape)
print("Saved as:", OUTPUT_FILE)


Cleaning complete
Final dataset shape: (8546, 11)
Saved as: output/clean_cafe_sales.csv
